In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm
import category_encoders as ce

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# Data preparation

In [ ]:
data0 = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
data0[0:2].T

In [ ]:
data0.columns

In [ ]:
data0.info()

In [ ]:
fig, ax = plt.subplots(figsize=(16,4)) 
sns.histplot(data0['Pawpularity'], label='Pawpularity', ax=ax, color='C1',bins=100) 
ax.legend() 
ax.grid()

In [ ]:
#set new parameter
for i in range(len(data0)):
    data0.loc[i,'sum']=data0.iloc[i,1:-2].sum()
data0['sum']

In [ ]:
data1=data0.drop('Id',axis=1)
data1[0:2].T

In [ ]:
data1.columns

# Target setting

In [ ]:
target=['Pawpularity']
datay=data1[target]
datax=data1.drop([target[0]],axis=1)

In [ ]:
print(datay[0:2].T)
print()
print(datax[0:2].T)

In [ ]:
train=np.array(datax)
trainy=np.array(datay)
print(train.shape)
print(trainy.shape)

In [ ]:
df_columns = list(datax.columns)
print(df_columns)

In [ ]:
train_df=pd.DataFrame(train)
#test_df=pd.DataFrame(test)

In [ ]:
train_df.columns=df_columns
#test_df.columns=df_columns

In [ ]:
def create_numeric_feature(input_df):
    use_columns = df_columns 
    return input_df[use_columns].copy()

In [ ]:
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [ ]:
from tqdm import tqdm

def to_feature(input_df):

    processors = [
        create_numeric_feature,
    ]
    
    out_df = pd.DataFrame()
    
    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
        
    return out_df

In [ ]:
train_feat_df = to_feature(train_df)
#test_feat_df = to_feature(test_df)

# Model

In [ ]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv, 
             params: dict=None, 
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)
        
        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
params = {
    'objective': 'rmse', 
    'learning_rate': .1,
    'reg_lambda': 1.,
    'reg_alpha': .1,
    'max_depth': 5, 
    'n_estimators': 10000, 
    'colsample_bytree': .5, 
    'min_child_samples': 10,
    'subsample_freq': 3,
    'subsample': .9,
    'importance_type': 'gain', 
    'random_state': 71,
    'num_leaves': 62
}

In [ ]:
y = trainy
print(y.shape)
print(y[0:3])

In [ ]:
ydf=pd.DataFrame(y)
ydf

In [ ]:
from sklearn.model_selection import KFold

for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    
    fig,ax = plt.subplots(figsize=(6,6))
    ax.set_title(target[i],fontsize=20)
    ax.set_ylabel('Predicted train '+target[i],fontsize=12)
    ax.set_xlabel('Actual train '+target[i],fontsize=12)
    ax.scatter(y,oof)


In [ ]:
print(target)

# Visualize Importance

In [ ]:
def visualize_importance(models, feat_train_df):

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    
    ax.tick_params(axis='x', rotation=0)
    #ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    
    return fig,ax

#fig, ax = visualize_importance(models, train_feat_df)

In [ ]:
for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    fig, ax = visualize_importance(models, train_feat_df)
    ax.set_title(target[i]+' Imortance',fontsize=20)


In [ ]:
fig,ax = plt.subplots(figsize=(6,6))
ax.set_title('Sum vs '+target[0],fontsize=20)
ax.set_ylabel('Sum',fontsize=12)
ax.set_xlabel(target[0],fontsize=12)
ax.scatter(data0[target[0]],data0['sum'])

In [ ]:
fig,ax = plt.subplots(figsize=(6,6))
ax.set_title('Near vs '+target[0],fontsize=20)
ax.set_ylabel('Near',fontsize=12)
ax.set_xlabel(target[0],fontsize=12)
ax.scatter(data0[target[0]],data0['Near'])


In [ ]:
print(data0['Pawpularity'][data0['Near']==1].mean())
print(data0['Pawpularity'][data0['Near']==0].mean())
# Importance of 'Near' is high, not big differnce in 'Pawpularity'

In [ ]:
n=len(data0)
print(n)
N=list(range(n//50))
random.seed(2021)
random.shuffle(N)

In [ ]:
train_dir='../input/petfinder-pawpularity-score/train'

In [ ]:
IMG=list()
PAW=list()
for i in tqdm(range(n//50)):
    file=data0.loc[i,'Id']
    path=os.path.join(train_dir,file+'.jpg')
    img1=cv2.imread(path)
    img2=cv2.resize(img1,dsize=(60,60),interpolation=cv2.INTER_CUBIC)
    IMG+=[img2]
    PAW+=[data0.loc[i,'Pawpularity']]

In [ ]:
fig, axs = plt.subplots(4,4,figsize=(14,14))
for i in range(16):
    r=i//4
    c=i%4
    ax=axs[r][c].axis("off")
    ax=axs[r][c].set_title('Paw: '+str(PAW[N[i]]))
    ax=axs[r][c].imshow(cv2.cvtColor(IMG[N[i]], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
data0d=data0[data0['sum']>5].reset_index(drop=True)
print(len(data0d))

In [ ]:
IMG1=list()
PAW1=list()
for i in tqdm(range(n//50)):
    file=data0d.loc[i,'Id']
    path=os.path.join(train_dir,file+'.jpg')
    img1=cv2.imread(path)
    img2=cv2.resize(img1,dsize=(60,60),interpolation=cv2.INTER_CUBIC)
    IMG1+=[img2]
    PAW1+=[data0d.loc[i,'Pawpularity']]

In [ ]:
fig, axs = plt.subplots(4,4,figsize=(14,14))
for i in range(16):
    r=i//4
    c=i%4
    ax=axs[r][c].axis("off")
    ax=axs[r][c].set_title('sum:>5, Paw:'+str(PAW1[N[i]]))
    ax=axs[r][c].imshow(cv2.cvtColor(IMG1[N[i]], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
data0['Near'].value_counts()

In [ ]:
data0a=data0[data0['Near']==1].reset_index(drop=True)
print(len(data0a))

In [ ]:
IMG2=list()
PAW2=list()
for i in tqdm(range(n//50)):
    file=data0a.loc[i,'Id']
    path=os.path.join(train_dir,file+'.jpg')
    img1=cv2.imread(path)
    img2=cv2.resize(img1,dsize=(60,60),interpolation=cv2.INTER_CUBIC)
    IMG2+=[img2]
    PAW2+=[data0a.loc[i,'Pawpularity']]

In [ ]:
fig, axs = plt.subplots(4,4,figsize=(14,14))
for i in range(16):
    r=i//4
    c=i%4
    ax=axs[r][c].axis("off")
    ax=axs[r][c].set_title('Near:1, Paw:'+str(PAW2[N[i]]))
    ax=axs[r][c].imshow(cv2.cvtColor(IMG2[N[i]], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
data0b=data0[data0['Pawpularity']==100].reset_index(drop=True)
print(len(data0b))

In [ ]:
IMG3=list()
PAW3=list()
for i in tqdm(range(n//50)):
    file=data0b.loc[i,'Id']
    path=os.path.join(train_dir,file+'.jpg')
    img1=cv2.imread(path)
    img2=cv2.resize(img1,dsize=(60,60),interpolation=cv2.INTER_CUBIC)
    IMG3+=[img2]
    PAW3+=[data0b.loc[i,'Pawpularity']]

In [ ]:
fig, axs = plt.subplots(4,4,figsize=(14,14))
for i in range(16):
    r=i//4
    c=i%4
    ax=axs[r][c].axis("off")
    ax=axs[r][c].set_title('Paw:'+str(PAW3[N[i]]))
    ax=axs[r][c].imshow(cv2.cvtColor(IMG3[N[i]], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
data0c=data0[data0['Pawpularity']<10].reset_index(drop=True)
print(len(data0c))

In [ ]:
IMG4=list()
PAW4=list()
for i in tqdm(range(n//50)):
    file=data0c.loc[i,'Id']
    path=os.path.join(train_dir,file+'.jpg')
    img1=cv2.imread(path)
    img2=cv2.resize(img1,dsize=(60,60),interpolation=cv2.INTER_CUBIC)
    IMG4+=[img2]
    PAW4+=[data0c.loc[i,'Pawpularity']]

In [ ]:
fig, axs = plt.subplots(4,4,figsize=(14,14))
for i in range(16):
    r=i//4
    c=i%4
    ax=axs[r][c].axis("off")
    ax=axs[r][c].set_title('Paw:'+str(PAW4[N[i]]))
    ax=axs[r][c].imshow(cv2.cvtColor(IMG4[N[i]], cv2.COLOR_BGR2RGB))
plt.show()